## Data Loading and Data Prep

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [2]:
df = pd.read_csv('stud.csv')

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [5]:
X,Y = df.drop('math_score', axis=1), df['math_score']
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [6]:
Y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [11]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((800, 7), (200, 7), (800,), (200,))

In [16]:
num_features = [feature for feature in X.columns if X[feature].dtype != 'O']
cat_features = [feature for feature in X.columns if X[feature].dtype == 'O']
len(num_features), len(cat_features)

(2, 5)

In [17]:
X_train_enc = pd.get_dummies(X_train, columns=cat_features).astype('int')
X_test_enc = pd.get_dummies(X_test, columns=cat_features).astype('int')

In [18]:
X_train_enc.head()

,reading_score,writing_score,gender_female,gender_male,race_ethnicity_group A,race_ethnicity_group B,race_ethnicity_group C,race_ethnicity_group D,race_ethnicity_group E,parental_level_of_education_associate's degree,parental_level_of_education_bachelor's degree,parental_level_of_education_high school,parental_level_of_education_master's degree,parental_level_of_education_some college,parental_level_of_education_some high school,lunch_free/reduced,lunch_standard,test_preparation_course_completed,test_preparation_course_none
29,70,75,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1
535,83,83,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0
695,89,86,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1
557,67,66,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
836,64,57,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1


In [20]:
scaler = StandardScaler().set_output(transform="pandas")
scaler.fit(X_train_enc[num_features])
X_train_final = scaler.transform(X_train_enc[num_features])
X_test_final = scaler.transform(X_test_enc[num_features])

In [21]:
X_train_final.head()

,reading_score,writing_score
29,0.030791,0.434053
535,0.930290,0.964701
695,1.345443,1.163694
557,-0.176786,-0.162925
836,-0.384363,-0.759904


In [24]:
X_train = pd.concat([X_train_enc.drop(num_features, axis=1),X_train_final], axis=1)
X_test = pd.concat([X_test_enc.drop(num_features, axis=1),X_test_final], axis=1)

In [25]:
X_train.head()

,gender_female,gender_male,race_ethnicity_group A,race_ethnicity_group B,race_ethnicity_group C,race_ethnicity_group D,race_ethnicity_group E,parental_level_of_education_associate's degree,parental_level_of_education_bachelor's degree,parental_level_of_education_high school,parental_level_of_education_master's degree,parental_level_of_education_some college,parental_level_of_education_some high school,lunch_free/reduced,lunch_standard,test_preparation_course_completed,test_preparation_course_none,reading_score,writing_score
29,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0.030791,0.434053
535,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0.930290,0.964701
695,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1.345443,1.163694
557,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,-0.176786,-0.162925
836,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,-0.384363,-0.759904


In [26]:
X_test.head()

,gender_female,gender_male,race_ethnicity_group A,race_ethnicity_group B,race_ethnicity_group C,race_ethnicity_group D,race_ethnicity_group E,parental_level_of_education_associate's degree,parental_level_of_education_bachelor's degree,parental_level_of_education_high school,parental_level_of_education_master's degree,parental_level_of_education_some college,parental_level_of_education_some high school,lunch_free/reduced,lunch_standard,test_preparation_course_completed,test_preparation_course_none,reading_score,writing_score
521,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1.137866,1.031032
737,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,-0.245978,0.301391
740,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0.238367,0.235060
660,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0.515136,0.301391
411,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0.930290,0.633046


In [27]:
X_train.shape, X_test.shape

((800, 19), (200, 19))

## Model Training and Evaluation

In [36]:
def evaluation_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true,predicted)
    return mae, rmse, r2

In [37]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

In [38]:
model_list = []
r2_list =[]

In [39]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    model_train_mae , model_train_rmse, model_train_r2 = evaluation_model(Y_train, y_train_pred)
    model_test_mae , model_test_rmse, model_test_r2 = evaluation_model(Y_test, y_test_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print('=============================================')
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3257
- Mean Absolute Error: 4.2698
- R2 Score: 0.8742
Model performance for Test set
- Root Mean Squared Error: 5.4212
- Mean Absolute Error: 4.2346
- R2 Score: 0.8792


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5925
- Mean Absolute Error: 5.2053
- R2 Score: 0.8072
Model performance for Test set
- Root Mean Squared Error: 6.5173
- Mean Absolute Error: 5.1557
- R2 Score: 0.8254


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.3233
- Mean Absolute Error: 4.2650
- R2 Score: 0.8743
Model performance for Test set
- Root Mean Squared Error: 5.3904
- Mean Absolute Error: 4.2111
- R2 Score: 0.8806


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.6941
- Mean Absolute Error: 4.5025
- R2 Score: 0.8562
Model performance for Test set
- Root Mean Squared Error: 7.2148
- Mean Absolute Error: 5.5840
- R2 Score: 0.7861


Decision Tre

In [40]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Ridge,0.880592
0,Linear Regression,0.879223
7,CatBoosting Regressor,0.851831
5,Random Forest Regressor,0.850647
8,AdaBoost Regressor,0.850148
6,XGBRegressor,0.827797
1,Lasso,0.825446
3,K-Neighbors Regressor,0.786089
4,Decision Tree,0.710588
